**Table of contents**<a id='toc0_'></a>    
- [N2 Excited State Data](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[N2 Excited State Data](#toc0_)
**first testing if jupyer is a good notetaking tool**


| state  | avdz             | avtz             |
| ------ | ---------------- | ---------------- |
| gs     | -109.4869509(85) | -109.534973(17)* |
| 1Pig   | -109.145116(82)  | -109.19543(16)** |
| 1Sigu- | -109.12030(20)   |                  |
| 1Delu  | -109.107187(52)  |                  |
| 3Sigu+ | -109.201567(40)  |                  |
| 3Pig   | -109.192041(41)  |                  |
| 3Delu  | -109.156050(34)  |                  |

just use pandoc style citations [@haupt2022; @laughlinTheory2000] and worry about matching citations when building in LaTeX (use pandoc to convert these)

!cat table.md


In [24]:
from IPython.display import Markdown, display
import pandas as pd
import uncertainties
from uncertainties import ufloat_fromstr
import string



display_csv = lambda f: display(Markdown(pd.read_csv(f,comment="#").to_markdown(index=False)))
display_md = lambda f: display(Markdown(open(f).read()))

Result = 1.2(5)


In [25]:
# # display(Markdown("tmp.md"))
# display(Markdown(pd.read_csv("tmp.csv").to_markdown(index=False)))
display_csv("N2_states.csv")
# display_md("N2_states.md")

|  state     |  avdz                |  avtz           |
|:-----------|:---------------------|:----------------|
| gs         | -109.4869509(85)     | -109.534973(17) |
| 1Pig       | -109.145116(82)      | nan             |
| 1Sigu-     | -109.12030(20)       | nan             |
| 1Delu      | -109.107187(52)      | nan             |
| 3Sigu+     | -109.201567(40)      | nan             |
| 3Pig       | -109.192041(41)      | nan             |
| 3Delu      | -109.156050(34)      | nan             |

In [130]:
import pandas as pd

from uncertainties import unumpy
from uncertainties import ufloat, ufloat_fromstr
from uncertainties.core import UFloat
from math import isnan

def format_ufloat(x):
    if isinstance(x, type(ufloat(1, 1))):
        return f"{x:+.1uS}"  # Uses the shorthand format
    return x

def col2ufloat(col):
    return [ufloat_fromstr("nan") if pd.isna(x) else ufloat_fromstr(x) for x in col]

# read in data
df = pd.read_csv("N2_states.csv", comment='#', skipinitialspace=True)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
df.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too

# handle NaNs as ufloats as well
df["avdz"] = col2ufloat(df["avdz"])
df["avtz"] = col2ufloat(df["avtz"])

# NOTE this formatting converts to strings!
def display_formatted(df):
    # Create a copy to avoid modifying original
    display_df = df.copy()

    # Format ufloat columns
    for col in df.columns:
        if isinstance(df[col].iloc[0], UFloat):
            display_df[col] = df[col].apply(lambda x: f"{x:+.1uS}")

    display(display_df)

display_formatted(df)
type(df["avdz"][1])

,state,avdz,avtz
0,gs,-109.486951(8),-109.53497(2)
1,1Pig,-109.14512(8),+nan(1)
2,1Sigu-,-109.1203(2),+nan(1)
3,1Delu,-109.10719(5),+nan(1)
4,3Sigu+,-109.20157(4),+nan(1)
5,3Pig,-109.19204(4),+nan(1)
6,3Delu,-109.15605(3),+nan(1)


uncertainties.core.Variable

In [131]:
# Get gs values as a series
gs_values = df[df['state'] == 'gs'].iloc[0]
display(gs_values)

# Create new dataframe excluding gs
diff_df = df[df['state'] != 'gs'].copy()

cols_to_diff = df.columns[1:]  # All columns except first
for col in cols_to_diff:
    # display(diff_df[col])
    # display(gs_values[col])
    diff_df[col] = diff_df[col] - gs_values[col]
display_formatted(diff_df)
type(diff_df["avdz"].iloc[0])
# type(ufloat(1, 1))

state                        gs
avdz     -109.486951+/-0.000008
avtz     -109.534973+/-0.000017
Name: 0, dtype: object

,state,avdz,avtz
1,1Pig,+0.34183(8),+nan(1)
2,1Sigu-,+0.3667(2),+nan(1)
3,1Delu,+0.37976(5),+nan(1)
4,3Sigu+,+0.28538(4),+nan(1)
5,3Pig,+0.29491(4),+nan(1)
6,3Delu,+0.33090(4),+nan(1)


uncertainties.core.AffineScalarFunc

In [ ]:
diff_df